#Container Networking¶


In [1]:
!docker ps -a


CONTAINER ID        IMAGE                COMMAND                  CREATED             STATUS                     PORTS                    NAMES
6cc14daab209        node:12-alpine       "docker-entrypoint.s…"   26 hours ago        Up 26 hours                0.0.0.0:3000->3000/tcp   lucid_kepler
658bed8ba63f        docker101tutorial    "/docker-entrypoint.…"   3 days ago          Up 3 days                  0.0.0.0:82->80/tcp       docker-tutorial2
08b8f5391f27        docker101tutorial    "/docker-entrypoint.…"   3 days ago          Created                                             docker-tutorial
11c586eda298        alpine/git           "git --help"             3 days ago          Exited (0) 3 days ago                               hossein_repo6
b5f82006a8af        busybox              "echo 'hello from bu…"   4 days ago          Exited (0) 4 days ago                               elated_clarke
1c45cf2e260d        busybox              "sh"                     4 weeks ago         Exited

There are two ways to put a container on a network:

 1) Assign it at start 

 2) connect an existing container.
 
For now, we will create the network first and attach the MySQL container at startup.

In [2]:
!docker network create todo-app


444d3f40851c2f6eb8226a4511b5ec4cb40e51dc45a5b577ed6dc8e75234a379


Start a MySQL container and attach it to the network. We're also going to define a few environment variables that the database will use to initialize the database (see the "Environment Variables" section in the MySQL Docker Hub listing).

https://hub.docker.com/_/mysql/

In [3]:
!docker run -d \
    --network todo-app --network-alias mysql \
    -v todo-mysql-data:/var/lib/mysql \
    -e MYSQL_ROOT_PASSWORD=secret \
    -e MYSQL_DATABASE=todos \
    mysql:5.7

Unable to find image 'mysql:5.7' locally
5.7: Pulling from library/mysql











Digest: sha256:14fd47ec8724954b63d1a236d2299b8da25c9bbb8eacc739bb88038d82da4919
Status: Downloaded newer image for mysql:5.7
b6d2821dd72ee55ca38546aada983a672c05290de618a8a5b8ca6de969c5300a


You'll notice we're using a volume named todo-mysql-data here and mounting it at /var/lib/mysql, which is where MySQL stores its data. However, we never ran a docker volume create command. Docker recognizes we want to use a named volume and creates one automatically for us.


You'll also see we specified the --network-alias flag. We'll come back to that in just a moment.

In [5]:
!docker ps -a

CONTAINER ID        IMAGE                COMMAND                  CREATED             STATUS                     PORTS                    NAMES
b6d2821dd72e        mysql:5.7            "docker-entrypoint.s…"   3 minutes ago       Up 3 minutes               3306/tcp, 33060/tcp      blissful_allen
6cc14daab209        node:12-alpine       "docker-entrypoint.s…"   26 hours ago        Up 26 hours                0.0.0.0:3000->3000/tcp   lucid_kepler
658bed8ba63f        docker101tutorial    "/docker-entrypoint.…"   3 days ago          Up 3 days                  0.0.0.0:82->80/tcp       docker-tutorial2
08b8f5391f27        docker101tutorial    "/docker-entrypoint.…"   3 days ago          Created                                             docker-tutorial
11c586eda298        alpine/git           "git --help"             3 days ago          Exited (0) 3 days ago                               hossein_repo6
b5f82006a8af        busybox              "echo 'hello from bu…"   4 days ago          Exite

In [6]:
!docker exec -it blissful_allen mysql -p


Enter password: 
ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)


When the password prompt comes up, type in secret. In the MySQL shell, list the databases and verify you see the todos database.


mysql> SHOW DATABASES;

In [8]:
!docker run -it --network todo-app nicolaka/netshoot


                    dP            dP                           dP   
                    88            88                           88   
88d888b. .d8888b. d8888P .d8888b. 88d888b. .d8888b. .d8888b. d8888P 
88'  `88 88ooood8   88   Y8ooooo. 88'  `88 88'  `88 88'  `88   88   
88    88 88.  ...   88         88 88    88 88.  .88 88.  .88   88   
dP    dP `88888P'   dP   `88888P' dP    dP `88888P' `88888P'   dP   
                                                                    

root @ / 
 [1] 🐳  → ^C


root @ / 
 [1] 🐳  → 

In [9]:
!docker ps -a

CONTAINER ID        IMAGE                COMMAND                  CREATED              STATUS                        PORTS                    NAMES
c4a7ca86b056        nicolaka/netshoot    "/bin/bash -l"           36 seconds ago       Exited (129) 34 seconds ago                            keen_bose
ba6eaa045e23        nicolaka/netshoot    "/bin/bash -l"           About a minute ago   Exited (129) 38 seconds ago                            hopeful_kepler
b6d2821dd72e        mysql:5.7            "docker-entrypoint.s…"   12 minutes ago       Up 12 minutes                 3306/tcp, 33060/tcp      blissful_allen
6cc14daab209        node:12-alpine       "docker-entrypoint.s…"   27 hours ago         Up 27 hours                   0.0.0.0:3000->3000/tcp   lucid_kepler
658bed8ba63f        docker101tutorial    "/docker-entrypoint.…"   3 days ago           Up 3 days                     0.0.0.0:82->80/tcp       docker-tutorial2
08b8f5391f27        docker101tutorial    "/docker-entrypoint.…"   3 days

Inside the container, we're going to use the dig command, which is a useful DNS tool. We're going to look up the IP address for the hostname mysql.


dig mysql

In the "ANSWER SECTION", you will see an A record for mysql that resolves to 172.23.0.2 (your IP address will most likely have a different value). While mysql isn't normally a valid hostname, Docker was able to resolve it to the IP address of the container that had that network alias (remember the --network-alias flag we used earlier?).

What this means is... our app only simply needs to connect to a host named mysql and it'll talk to the database! It doesn't get much simpler than that!


In [11]:
!ls
%cd ../../media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/app/

Desktop    Downloads  localperl  Pictures  snap       Videos
Documents  gopath     Music	 Public    Templates
/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/app


In [13]:
!docker run -dp 3000:3000 \
  -w /app -v "$(pwd):/app" \
  --network todo-app \
  -e MYSQL_HOST=mysql \
  -e MYSQL_USER=root \
  -e MYSQL_PASSWORD=secret \
  -e MYSQL_DB=todos \
  node:12-alpine \
  sh -c "yarn install && yarn run dev"

4f7f1f597c4b3101bb32f15066f866895b5b360d8809b2310115b5e720d5da7d


In [14]:
!docker ps -a

CONTAINER ID        IMAGE                COMMAND                  CREATED             STATUS                     PORTS                    NAMES
4f7f1f597c4b        node:12-alpine       "docker-entrypoint.s…"   32 seconds ago      Up 31 seconds              0.0.0.0:3000->3000/tcp   clever_leakey
bb07ab843c4d        node:12-alpine       "docker-entrypoint.s…"   4 minutes ago       Created                                             competent_kowalevski
4493421d5a28        nicolaka/netshoot    "/bin/bash -l"           14 minutes ago      Up 14 minutes                                       intelligent_pike
b6d2821dd72e        mysql:5.7            "docker-entrypoint.s…"   29 minutes ago      Up 29 minutes              3306/tcp, 33060/tcp      blissful_allen
658bed8ba63f        docker101tutorial    "/docker-entrypoint.…"   3 days ago          Up 3 days                  0.0.0.0:82->80/tcp       docker-tutorial2
11c586eda298        alpine/git           "git --help"             3 days ago      

In [15]:
!docker logs clever_leakey

yarn install v1.22.4
[1/4] Resolving packages...
[2/4] Fetching packages...
info fsevents@1.2.9: The platform "linux" is incompatible with this module.
info "fsevents@1.2.9" is an optional dependency and failed compatibility check. Excluding it from installation.
[3/4] Linking dependencies...
[4/4] Building fresh packages...
Done in 9.22s.
yarn run v1.22.4
$ nodemon src/index.js
[nodemon] 1.19.2
[nodemon] to restart at any time, enter `rs`
[nodemon] watching dir(s): *.*
[nodemon] starting `node src/index.js`
Waiting for mysql:3306.
Connected!
Connected to mysql db at host mysql
Listening on port 3000


Connect to the mysql database and prove that the items are being written to the database. Remember, the password is secret.


docker exec -it <mysql-container-id> mysql -p todos
And in the mysql shell, run the following:


mysql> select * from todo_items;

## Recap

At this point, we have an application that now stores its data in an external database running in a separate container. We learned a little bit about container networking and saw how service discovery can be performed using DNS.

But, there's a good chance you are starting to feel a little overwhelmed with everything you need to do to start up this application. We have to create a network, start containers, specify all of the environment variables, expose ports, and more! That's a lot to remember and it's certainly making things harder to pass along to someone else.

In the next section, we'll talk about Docker Compose. With Docker Compose, we can share our application stacks in a much easier way and let others spin them up with a single (and simple) command!

# Docker Compose

Installing Docker Compose¶
If you installed Docker Desktop/Toolbox for either Windows or Mac, you already have Docker Compose! Play-with-Docker instances already have Docker Compose installed as well. If you are on a Linux machine, you will need to install Docker Compose using the instructions here.

https://docs.docker.com/compose/install/

After installation, you should be able to run the following and see version information.


docker-compose version

## Creating our Compose File
At the root of the app project, create a file named docker-compose.yml.

In the compose file, we'll start off by defining the schema version. In most cases, it's best to use the latest supported version. You can look at the Compose file reference for the current schema versions and the compatibility matrix.


version: "3.7"

First, let's define the service entry and the image for the container. 

We can pick any name for the service. 

The name will automatically become a network alias, which will be useful when defining our MySQL service.

Typically, you will see the command close to the image definition, although there is no requirement on ordering. So, let's go ahead and move that into our file.

Let's migrate the -p 3000:3000 part of the command by defining the ports for the service. We will use the short syntax here, but there is also a more verbose long syntax available as well.

Next, we'll migrate both the working directory (-w /app) and the volume mapping (-v "$(pwd):/app") by using the working_dir and volumes definitions. Volumes also has a short and long syntax.

One advantage of Docker Compose volume definitions is we can use relative paths from the current directory.

```
services:
    app:
        image: node:12-alpine
        command: sh -c "yarn install && yarn run dev"
        ports:
            - 3000:3000
        working_dir: /app
        volumes:
            - ./:/app
        environment:
            MYSQL_HOST: mysql
            MYSQL_USER: root
            MYSQL_PASSWORD: secret
            MYSQL_DB: todos
```

Defining the MySQL Service


Now, it's time to define the MySQL service.

we'll define the volume mapping. When we ran the container with docker run, the named volume was created automatically. However, that doesn't happen when running with Compose. We need to define the volume in the top-level volumes: section and then specify the mountpoint in the service config. By simply providing only the volume name, the default options are used. There are many more options available though.
https://docs.docker.com/compose/compose-file/#volume-configuration-reference

```
services:
  app:
    image: node:12-alpine
    command: sh -c "yarn install && yarn run dev"
    ports:
      - 3000:3000
    working_dir: /app
    volumes:
      - ./:/app
    environment:
      MYSQL_HOST: mysql
      MYSQL_USER: root
      MYSQL_PASSWORD: secret
      MYSQL_DB: todos

  mysql:
    image: mysql:5.7
    volumes:
      - todo-mysql-data:/var/lib/mysql
    environment: 
      MYSQL_ROOT_PASSWORD: secret
      MYSQL_DATABASE: todos

volumes:
  todo-mysql-data:
```

Start up the application stack using the docker-compose up command. We'll add the -d flag to run everything in the background. (in the repo root directory)

```
docker-compose up -d
```

When we run this, we should see output like this:

```
Creating network "app_default" with the default driver
Creating volume "app_todo-mysql-data" with default driver
Creating app_app_1   ... done
Creating app_mysql_1 ... done
```

the volume was created as well as a network! By default, Docker Compose automatically creates a network specifically for the application stack (which is why we didn't define one in the compose file).

In [17]:
!ls
!docker-compose logs -f

^C
ERROR: Aborting.


When you're ready to tear it all down, simply run docker-compose down or hit the trash can on the Docker Dashboard for the entire app. The containers will stop and the network will be removed.

Removing Volumes

By default, named volumes in your compose file are NOT removed when running docker-compose down. If you want to remove the volumes, you will need to add the --volumes flag.

The Docker Dashboard does not remove volumes when you delete the app stack.

# Image Building

## Image Layering

you can look at what makes up an image.

Using the docker image history command, you can see the command that was used to create each layer within an image.

Use the docker image history command to see the layers in the getting-started image you created earlier in the tutorial.

In [18]:
!docker image history getting-started


IMAGE               CREATED             CREATED BY                                      SIZE                COMMENT
52a93e117f26        46 hours ago        /bin/sh -c #(nop)  CMD ["node" "src/index.js…   0B                  
267f3b5095e4        46 hours ago        /bin/sh -c yarn install --production            85.2MB              
2ee0676e1e28        46 hours ago        /bin/sh -c #(nop) COPY dir:e1b772069652eaa1b…   4.62MB              
e1a1be3e3f39        3 days ago          /bin/sh -c #(nop) WORKDIR /app                  0B                  
1f52b7199ba6        2 weeks ago         /bin/sh -c #(nop)  CMD ["node"]                 0B                  
<missing>           2 weeks ago         /bin/sh -c #(nop)  ENTRYPOINT ["docker-entry…   0B                  
<missing>           2 weeks ago         /bin/sh -c #(nop) COPY file:238737301d473041…   116B                
<missing>           2 weeks ago         /bin/sh -c apk add --no-cache --virtual .bui…   7.62MB              
<missing>   

In [19]:
!docker image history --no-trunc getting-started


IMAGE                                                                     CREATED             CREATED BY                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

`.dockerignore `files are an easy way to selectively copy only image relevant files. You can read more about this here. 

https://docs.docker.com/engine/reference/builder/#dockerignore-file

In this case, the `node_modules` folder should be omitted in the second `COPY `step because otherwise, it would possibly overwrite files which were created by the command in the RUN step. For further details on why this is recommended for Node.js applications and other best practices, have a look at their guide on Dockerizing a Node.js web app.

https://nodejs.org/en/docs/guides/nodejs-docker-webapp/


In [20]:
!docker build -t getting-started .


Sending build context to Docker daemon  4.661MB
Step 1/6 : FROM node:12-alpine
 ---> 1f52b7199ba6
Step 2/6 : WORKDIR /app
 ---> Using cache
 ---> e1a1be3e3f39
Step 3/6 : COPY package.json yarn.lock ./
 ---> b142822a548b
Step 4/6 : RUN yarn install --production
 ---> Running in 843db6f6cfbe
yarn install v1.22.4
[1/4] Resolving packages...
[2/4] Fetching packages...
info fsevents@1.2.9: The platform "linux" is incompatible with this module.
info "fsevents@1.2.9" is an optional dependency and failed compatibility check. Excluding it from installation.
[3/4] Linking dependencies...
[4/4] Building fresh packages...
Done in 7.18s.
Removing intermediate container 843db6f6cfbe
 ---> 230d25052e66
Step 5/6 : COPY . .
 ---> d36fe62d3d65
Step 6/6 : CMD ["node", "src/index.js"]
 ---> Running in 64998beb9135
Removing intermediate container 64998beb9135
 ---> 3868012c04e9
Successfully built 3868012c04e9
Successfully tagged getting-started:latest


You'll see that all layers were rebuilt. Perfectly fine since we changed the Dockerfile quite a bit.

Now, make a change to the src/static/index.html file (like change the <title> to say "The Awesome Todo App").

Build the Docker image now using docker build -t getting-started . again. This time, your output should look a little different.

you should notice that the build was MUCH faster! And, you'll see that steps 1-4 all have Using cache. So, hooray! We're using the build cache. Pushing and pulling this image and updates to it will be much faster as well.

# Multi-Stage Builds

multi-stage builds are an incredibly powerful tool to help use multiple stages to create an image.

There are several advantages for them:

* Separate build-time dependencies from runtime dependencies

* Reduce overall image size by shipping only what your app needs to run

##Maven/Tomcat Example
When building Java-based applications, a JDK is needed to compile the source code to Java bytecode. However, that JDK isn't needed in production. Also, you might be using tools like Maven or Gradle to help build the app. Those also aren't needed in our final image. Multi-stage builds help.


```
FROM maven AS build
WORKDIR /app
COPY . .
RUN mvn package


FROM tomcat
COPY --from=build /app/target/file.war /usr/local/tomcat/webapps 

```

In this example, we use one stage (called build) to perform the actual Java build using Maven. In the second stage (starting at FROM tomcat), we copy in files from the build stage. The final image is only the last stage being created (which can be overridden using the --target flag).

##React Example
When building React applications, we need a Node environment to compile the JS code (typically JSX), SASS stylesheets, and more into static HTML, JS, and CSS. If we aren't doing server-side rendering, we don't even need a Node environment for our production build. Why not ship the static resources in a static nginx container?


FROM node:12 AS build
WORKDIR /app
COPY package* yarn.lock ./
RUN yarn install
COPY public ./public
COPY src ./src
RUN yarn run build

FROM nginx:alpine
COPY --from=build /app/build /usr/share/nginx/html
Here, we are using a node:12 image to perform the build (maximizing layer caching) and then copying the output into an nginx container. Cool, huh?

#Cloud Native Computing Foundation Projects

The CNCF is a vendor-neutral home for various open-source projects, including Kubernetes, Prometheus, Envoy, Linkerd, NATS, and more! You can view the graduated and incubated projects here and the entire CNCF Landscape here. There are a LOT of projects to help solve problems around monitoring, logging, security, image registries, messaging, and more!

So, if you're new to the container landscape and cloud-native application development, welcome! Please connect to the community, ask questions, and keep learning! We're excited to have you!

https://www.cncf.io/projects/

https://landscape.cncf.io/

# Source


https://github.com/docker/getting-started